In [1]:
import pandas as pd
import numpy as np
import psycopg2
from sklearn.ensemble import IsolationForest
import joblib

In [2]:
# Connectting to our PostgresSQL

conn = psycopg2.connect(
    host="localhost",
    database="ehr_data",
    user="postgres",
    password="password"
)

In [3]:
# Load all observations with 'glucose' in description

df = pd.read_sql_query("""
    SELECT patient_id, value::float, unit, effective_date
    FROM observations
    WHERE LOWER(description) LIKE '%glucose%' AND value ~ '^[0-9.]+$'
    ORDER BY effective_date
""", conn)

conn.close()


print(f"✅ Loaded {len(df)} glucose-related observation records.")
print("🔍 Sample values:")
print(df.head())

✅ Loaded 2810 glucose-related observation records.
🔍 Sample values:
                             patient_id  value   unit      effective_date
0  1871d3bf-072c-aabf-d872-d5355a5196be  84.02  mg/dL 1981-09-08 16:03:42
1  1871d3bf-072c-aabf-d872-d5355a5196be  97.87  mg/dL 1981-09-26 17:14:53
2  1871d3bf-072c-aabf-d872-d5355a5196be  93.67  mg/dL 1982-03-25 16:14:53
3  1871d3bf-072c-aabf-d872-d5355a5196be  93.67  mg/dL 1982-03-26 04:23:24
4  1871d3bf-072c-aabf-d872-d5355a5196be  64.20  mg/dL 1982-09-22 18:02:10


C:\Users\kunal\AppData\Local\Temp\ipykernel_19384\3894802549.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""


In [4]:
# Drop nulls and reset index
df.dropna(subset=["value"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
# Feature engineering
X = df[["value"]]

# Train an Isolation Forest
model = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
model.fit(X)

# Predict to show anomaly rate
preds = model.predict(X)
n_anomalies = (preds == -1).sum()
print(f"⚠️ Detected {n_anomalies} anomalies out of {len(preds)} values.")

# Save model
joblib.dump(model, "models/glucose_model.pkl")
print("✅ Model trained and saved at train/models/glucose_model.pkl")

⚠️ Detected 141 anomalies out of 2810 values.
✅ Model trained and saved at train/models/glucose_model.pkl
